In [112]:
# In this notebook we will read the data from 2 pressuresensors that are connected to the same hub. 
# Next we will process this data and calculate the speed of the liquid.
# Lastly we will create a virtual (speed) sensor and push virtual data to the database.

In [113]:
# Run this line if you don't have the 'requests' package installed.
# !pip3 install requests

In [114]:
# Run this line if you don't have the 'pandas' package installed.
# !pip3 install pandas

In [115]:
# Run this line if you don't have the 'matplotlib' package installed.
# !pip3 install matplotlib

In [182]:
import math 
import requests
import pandas as pd

In [117]:
# Fill in your API server ip.
serverIp = "192.168.0.28"

In [118]:
# Fill in your API server port.
serverPort = "3030"

In [119]:
url = "http://"+serverIp+":"+serverPort+"/api/v1/data"

In [120]:
# Choose the hub that includes the pressuresensors
hubId = "TEST_SPEED_HUB"

getSpecificData = {
    "hubId": hubId,
}

In [121]:
r = requests.get(url, getSpecificData)
json = r.json()
# json

In [122]:
# sensorData is a list of dicts.
sensorData = json['data']
# sensorData

In [123]:
df = pd.DataFrame(sensorData)

# Show all columns
#df

# Show important columns
df[["hubId", "sensorId", "value", "createdAt"]]

,hubId,sensorId,value,createdAt
0,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:49.844Z
1,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:50.427Z
2,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:51.833Z
3,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:52.464Z
4,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:05.167Z
5,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:05.710Z
6,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:06.139Z
7,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:06.488Z


In [124]:
# Fill in which pressureSensor and hub you want a dataframe of
pressureSensorId_1 = "PS0"
pressureSensorId_2 = "PS1"

In [165]:
pressureSensor_1_df = df[df["sensorId"] == pressureSensorId_1]
pressureSensor_2_df = df[df["sensorId"] == pressureSensorId_2]


In [166]:
# Show sensor 1 df
pressureSensor_1_df

,_id,hubId,sensorId,value,createdAt,updatedAt,__v
0,60b8173d2ac5190013713e06,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:49.844Z,2021-06-02T23:41:49.844Z,0
1,60b8173e2ac5190013713e07,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:50.427Z,2021-06-02T23:41:50.427Z,0
2,60b8173f2ac5190013713e08,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:51.833Z,2021-06-02T23:41:51.833Z,0
3,60b817402ac5190013713e09,TEST_SPEED_HUB,PS0,1500,2021-06-02T23:41:52.464Z,2021-06-02T23:41:52.464Z,0


In [167]:
# Show sensor 2 df
pressureSensor_2_df

,_id,hubId,sensorId,value,createdAt,updatedAt,__v
4,60b8174d2ac5190013713e0a,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:05.167Z,2021-06-02T23:42:05.167Z,0
5,60b8174d2ac5190013713e0b,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:05.710Z,2021-06-02T23:42:05.710Z,0
6,60b8174e2ac5190013713e0c,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:06.139Z,2021-06-02T23:42:06.139Z,0
7,60b8174e2ac5190013713e0d,TEST_SPEED_HUB,PS1,2000,2021-06-02T23:42:06.488Z,2021-06-02T23:42:06.488Z,0


In [243]:
virtualSpeedSensorData = {'hubId': hubId, 'sensorId': "VSS", 'value': []} 

In [244]:
total_rows_sensor_1 = len(pressureSensor_1_df.index)
total_rows_sensor_2 = len(pressureSensor_2_df.index)

In [257]:
for x in range(total_rows_sensor_1):
    temp_value_1 = pressureSensor_1_df['value'][x]
    temp_value_2 = pressureSensor_2_df['value'][x+len(pressureSensor_1_df.index)]
    
    rhoFluid = 1000 # kg/m3
    area_point_1 = math.pi * ((9.6 / 1000) ** 2)
    area_point_2 = math.pi * ((19.2 / 1000) ** 2)
    
    # deltaP = P1 - P2
    deltaPressure = temp_value_1 - temp_value_2
    
    numerator = ((2 * deltaPressure) / rhoFluid)
    denominator = (1 - ((area_point_1 ** 2)/(area_point_2 ** 2)))
    
    speedPoint_1 = math.sqrt(abs(numerator)/denominator)
    
    virtualSpeedSensorData['value'].append(speedPoint_1)

In [246]:
virtualSpeedSensorData

{'hubId': 'TEST_SPEED_HUB',
 'sensorId': 'VSS',
 'value': [1.0327955589886444,
  1.0327955589886444,
  1.0327955589886444,
  1.0327955589886444]}

In [247]:
df = pd.DataFrame(virtualSpeedSensorData)  
df

,hubId,sensorId,value
0,TEST_SPEED_HUB,VSS,1.032796
1,TEST_SPEED_HUB,VSS,1.032796
2,TEST_SPEED_HUB,VSS,1.032796
3,TEST_SPEED_HUB,VSS,1.032796


In [263]:
for x in range(len(df.index)):
    payload = {
      "hubId": hubId,
      "sensorId": "speedSensor_VIRTUAL",
      "value": df.value[x]
    }
    
    r = requests.post(url, payload)
    json = r.json()
    print(json)

{'_id': '60b8f54a2ac5190013713e33', 'hubId': 'TEST_SPEED_HUB', 'sensorId': 'speedSensor_VIRTUAL', 'value': 1.0327955589886444, 'createdAt': '2021-06-03T15:29:14.497Z', 'updatedAt': '2021-06-03T15:29:14.497Z', '__v': 0}
{'_id': '60b8f54a2ac5190013713e34', 'hubId': 'TEST_SPEED_HUB', 'sensorId': 'speedSensor_VIRTUAL', 'value': 1.0327955589886444, 'createdAt': '2021-06-03T15:29:14.515Z', 'updatedAt': '2021-06-03T15:29:14.515Z', '__v': 0}
{'_id': '60b8f54a2ac5190013713e35', 'hubId': 'TEST_SPEED_HUB', 'sensorId': 'speedSensor_VIRTUAL', 'value': 1.0327955589886444, 'createdAt': '2021-06-03T15:29:14.525Z', 'updatedAt': '2021-06-03T15:29:14.525Z', '__v': 0}
{'_id': '60b8f54a2ac5190013713e36', 'hubId': 'TEST_SPEED_HUB', 'sensorId': 'speedSensor_VIRTUAL', 'value': 1.0327955589886444, 'createdAt': '2021-06-03T15:29:14.533Z', 'updatedAt': '2021-06-03T15:29:14.533Z', '__v': 0}


In [112]:
# End of file